<a href="https://colab.research.google.com/github/giangdip2410/Learn-NLP/blob/master/SentimentAnalysis_Dense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip "/content/drive/My Drive/SyncPC/Deep learning/IMDB Dataset.csv.zip" 

Archive:  /content/drive/My Drive/SyncPC/Deep learning/IMDB Dataset.csv.zip
replace IMDB Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: IMDB Dataset.csv        


In [0]:
import pandas as pd
data_pd = pd.read_csv("IMDB Dataset.csv")

In [0]:
#Using bag of word vectors
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=200)#,ngram_range=(1,2),stop_words=["the","that"])
X = vectorizer.fit_transform(data_pd["review"].values)
X = X.toarray()

In [0]:
data_pd["label"] = data_pd["sentiment"].map({'positive': 1, 'negative': 0})
data_pd.head()
y = data_pd["label"].values

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)


print (X_train.shape,X_test.shape, X_val.shape)

(20000, 200) (25000, 200) (5000, 200)


In [0]:
from keras.layers import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint


input_layer = Input(shape=(200,))
dense_layer = Dense(1, activation="sigmoid")(input_layer)

model = Model(inputs=input_layer, outputs=dense_layer)
model.compile(loss='binary_crossentropy', optimizer="adam",metrics=["accuracy"])
mc = ModelCheckpoint("best_checkpoint.h5", monitor="val_acc" , save_best_only=True, save_weights_only=True)

model.fit(X_train,y_train,validation_data=(X_val,y_val), epochs= 20, batch_size=20, callbacks=[mc])

Using TensorFlow backend.







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 20000 samples, validate on 5000 samples
Epoch 1/20





20000/20000 [==============================] - 1s 69us/step - loss: 0.6219 - acc: 0.6748 - val_loss: 0.5319 - val_acc: 0.7448
Epoch 2/20
20000/20000 [==============================] - 1s 63us/step - loss: 0.5076 - acc: 0.7577 - val_loss: 0.5041 - val_acc: 0.7602
Epoch 3/20
20000/20000 [==============================] - 1s 67us/step - loss: 0.4924 - acc: 0.7686 - val_loss: 0.5085 - val_acc: 0.7624
Epoch 4/20
20000/20000 [==============================] - 1s 64us/step - loss: 0.4890 - acc: 0.7687 - val_loss: 0.5085 - val_acc: 0.7608
Epoch 5/20
20000/20000 [==============================] - 1s 69us/step - loss: 0.4885 - acc: 0.7689 - val_loss: 0.5007 - val_acc: 0.7606
Epoch 6/20
20000/20000 [==============================] - 1s 64us/step - loss: 0.4881 - acc: 0.7710 - val_loss: 0.5000 - val_acc: 0.7618
Epoch 7/20
20000/20000

In [0]:
model.load_weights("best_checkpoint.h5")

In [0]:
y_test_pred = model.predict(X_test)
y_test_pred = [1 if pred > 0.5 else 0 for pred in y_test_pred ]

In [0]:
import numpy as np
print("Acc:",(1 - np.sum(np.abs(y_test_pred- y_test))/len(y_test_pred)))

Acc: 0.76032
